In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('./TempSense: Temperature and Sensor Data/Data.csv')

In [3]:
df

,Temperature,Sensor1,Sensor2,Sensor3,Sensor4,Sensor5
0,25.431489,0.677916,0.208399,0.033098,0.145758,0.291933
1,24.486056,0.801851,0.445989,0.869082,0.499015,0.531490
2,20.055087,0.781817,0.040596,0.205773,0.517225,0.598457
3,22.781690,0.651030,0.316018,0.945797,0.433242,0.378673
4,25.292826,0.292357,0.893463,0.289188,0.490347,0.012502
...,...,...,...,...,...,...
3452,21.254379,0.826416,0.198606,0.660799,0.049597,0.916904
3453,24.430966,0.286158,0.976771,0.632652,0.931474,0.149118
3454,28.474669,0.815359,0.450553,0.566098,0.443988,0.552319
3455,28.870984,0.217001,0.357490,0.552558,0.687092,0.602502


# EDA

In [4]:
df.isnull().sum()

Temperature    0
Sensor1        0
Sensor2        0
Sensor3        0
Sensor4        0
Sensor5        0
dtype: int64

#### Hurray!! We are lucky Here

In [5]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Temperature,3457.0,24.989396,2.898912,18.982907,22.542058,24.972157,27.396090,31.739198
Sensor1,3457.0,0.502612,0.290374,0.000243,0.252100,0.504355,0.758330,0.999787
Sensor2,3457.0,0.496721,0.293774,0.000388,0.240042,0.495343,0.750479,0.999965
Sensor3,3457.0,0.497600,0.287507,0.000091,0.249052,0.506430,0.750505,0.999586
Sensor4,3457.0,0.495230,0.287608,0.000244,0.251254,0.491202,0.741194,0.999667
Sensor5,3457.0,0.496095,0.289214,0.000254,0.241400,0.494519,0.748835,0.999862


# Pre-processing

In [8]:
X = df.drop('Temperature',axis=1)
y = df['Temperature']

In [6]:
from sklearn.preprocessing import PolynomialFeatures

In [7]:
poly = PolynomialFeatures(degree = 3)

In [9]:
scaled_X = poly.fit_transform(X)

In [10]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(scaled_X,y,test_size=0.2,random_state=42)

# Model Training

In [16]:
from sklearn.linear_model import ElasticNet

In [17]:
from sklearn.model_selection import GridSearchCV

In [18]:
ela = ElasticNet()

In [20]:
ela.fit(X_train,y_train)

ElasticNet()

In [32]:
ela.get_params()

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'l1_ratio': 0.5,
 'max_iter': 1000,
 'positive': False,
 'precompute': False,
 'random_state': None,
 'selection': 'cyclic',
 'tol': 0.0001,
 'warm_start': False}

In [21]:
ypred = ela.predict(X_test)

# Results

In [22]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [23]:
mae = mean_absolute_error(y_test,ypred)

In [24]:
rmse = mean_squared_error(y_test,ypred)**0.5

In [25]:
y_test.mean()

24.904570093956348

In [28]:
((mae/y_test.mean())*100),((rmse/y_test.mean())*100)

(9.801137206540172, 11.409488327445565)

# Tuning Parameters

In [35]:
param = {
        'l1_ratio':[0,0.5,0.75,1],
        'tol':[0.0001,0.001,0.00001],
        'random_state':[42],
}

In [36]:
grid = GridSearchCV(estimator=ela,param_grid=param)

In [37]:
grid.fit(X_train,y_train)

/home/vip/Softwares/envs/dl/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.389e+03, tolerance: 1.879e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/home/vip/Softwares/envs/dl/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.551e+03, tolerance: 1.912e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers 

GridSearchCV(estimator=ElasticNet(),
             param_grid={'l1_ratio': [0, 0.5, 0.75, 1], 'random_state': [42],
                         'tol': [0.0001, 0.001, 1e-05]})

In [39]:
ypred1 = grid.predict(X_test)

In [40]:
mae1 = mean_absolute_error(y_test,ypred1)
rmse1 = mean_squared_error(y_test,ypred1)**0.5
((mae1/y_test.mean())*100),((rmse1/y_test.mean())*100)

(9.801137206540172, 11.409488327445565)

In [48]:
param1 = {
        'l1_ratio':[0.25,0.5,0.6],
        'tol':[0.0001,0.001,0.00001],
        'random_state':[42],
        #'n_alphas':[100,200,50],
        'max_iter':[1000,2000,10000]
}

In [49]:
elastic = GridSearchCV(estimator=ela,param_grid=param1)

In [50]:
elastic.fit(X_train,y_train)

GridSearchCV(estimator=ElasticNet(),
             param_grid={'l1_ratio': [0.25, 0.5, 0.6],
                         'max_iter': [1000, 2000, 10000], 'random_state': [42],
                         'tol': [0.0001, 0.001, 1e-05]})

In [51]:
ypred2 = elastic.predict(X_test)

In [52]:
mae2 = mean_absolute_error(y_test,ypred2)
rmse2 = mean_squared_error(y_test,ypred2)**0.5
((mae2/y_test.mean())*100),((rmse2/y_test.mean())*100)

(9.801137206540172, 11.409488327445565)

In [54]:
elastic.best_params_

{'l1_ratio': 0.25, 'max_iter': 1000, 'random_state': 42, 'tol': 0.0001}

In [67]:
# Changing Input Ploynomial Degree

In [55]:
poly1 = PolynomialFeatures(degree = 5)

In [56]:
scaled_X1 = poly1.fit_transform(X)

In [57]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(scaled_X1,y,test_size=0.2,random_state=42)

In [58]:
ela.fit(X_train,y_train)

ElasticNet()

In [60]:
ypred0 = ela.predict(X_test)
mae0 = mean_absolute_error(y_test,ypred0)
rmse0 = mean_squared_error(y_test,ypred0)**0.5
((mae0/y_test.mean())*100),((rmse0/y_test.mean())*100)

(9.801137206540172, 11.409488327445565)

In [61]:
grid.fit(X_train,y_train)

/home/vip/Softwares/envs/dl/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.389e+03, tolerance: 1.879e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/home/vip/Softwares/envs/dl/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.551e+03, tolerance: 1.912e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers 

GridSearchCV(estimator=ElasticNet(),
             param_grid={'l1_ratio': [0, 0.5, 0.75, 1], 'random_state': [42],
                         'tol': [0.0001, 0.001, 1e-05]})

In [62]:
ypred01 = grid.predict(X_test)
mae01 = mean_absolute_error(y_test,ypred01)
rmse01 = mean_squared_error(y_test,ypred01)**0.5
((mae01/y_test.mean())*100),((rmse01/y_test.mean())*100)

(9.801137206540172, 11.409488327445565)

In [63]:
elastic.fit(X_train,y_train)

GridSearchCV(estimator=ElasticNet(),
             param_grid={'l1_ratio': [0.25, 0.5, 0.6],
                         'max_iter': [1000, 2000, 10000], 'random_state': [42],
                         'tol': [0.0001, 0.001, 1e-05]})

In [64]:
ypred02 = elastic.predict(X_test)
mae02 = mean_absolute_error(y_test,ypred02)
rmse02 = mean_squared_error(y_test,ypred02)**0.5
((mae02/y_test.mean())*100),((rmse02/y_test.mean())*100)

(9.801137206540172, 11.409488327445565)

### Well After Changing all the parameters result doesn't vary So finilazing First Model

In [65]:
import joblib

In [66]:
joblib.dump(ela,'Model.pkl')

['Model.pkl']